# Looking at output of VASP calculations to analyse accuracy of GGA + correction vs HSE optics spectra

In [2]:
from pymatgen.io.vasp import Vasprun
from vasppy.optics import absorption_coefficient as ac
import numpy as np
import matplotlib.pyplot as plt
from vasprun import vasprun
from pymatgen.electronic_structure.bandstructure import BandStructure
from SLME import Efficiency
import json
import pandas as pd
import re
import sys
import os

In [3]:
def loadjson(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

In [4]:
def dumpjson(data,filename):
    with open(filename, 'w+') as f:
        json.dump(data, f)

In [5]:
def shifter(spectrum,diff):
    if len(spectrum) == 100:
        new_alphas = np.append(np.array([0]),spectrum)
    else:
        new_alphas = spectrum
    alphas = np.array(new_alphas)
    Es = np.linspace(0,5,101)
    shifted_Es = Es+diff
    with_zero = np.insert(shifted_Es,0,0)
    right_length = np.delete(with_zero,-1)
    new_abs=np.interp(Es,right_length,alphas)
    return new_abs

In [ ]:
def extract_cpu_time(outcar_path):
    """
    Extract the total CPU time from a VASP OUTCAR file.

    Args:
        outcar_path (str): Path to the OUTCAR file

    Returns:
        float: Total CPU time in seconds, or None if not found
    """
    # Check if file exists
    if not os.path.isfile(outcar_path):
        print(f"Error: File {outcar_path} not found.")
        return None

    # Pattern to match the CPU time
    pattern = r"Total CPU time used \(sec\):\s*(\d+\.\d+)"

    # Read the file from the end since the CPU time is usually at the end
    with open(outcar_path, 'r') as file:
        # Read the last 50 lines which should contain the CPU time
        lines = file.readlines()
        last_lines = lines[-50:] if len(lines) > 50 else lines

        # Join the last lines into a single string
        text = ''.join(last_lines)

        # Search for the pattern
        match = re.search(pattern, text)

        if match:
            cpu_time = float(match.group(1))
            return cpu_time
        else:
            # If not found in the last 50 lines, search the entire file
            file.seek(0)
            text = file.read()
            match = re.search(pattern, text)

            if match:
                cpu_time = float(match.group(1))
                return cpu_time
            else:
                print("CPU time not found in the OUTCAR file.")
                return None

    if len(sys.argv) > 1:
        outcar_path = sys.argv[1]
    else:
        outcar_path = input("Enter the path to the OUTCAR file: ")

    cpu_time = extract_cpu_time(outcar_path)

    if cpu_time is not None:
        print(f"Total CPU time: {cpu_time} seconds")
        # Convert to hours, minutes, seconds for better readability
        hours = int(cpu_time // 3600)
        minutes = int((cpu_time % 3600) // 60)
        seconds = cpu_time % 60
        print(f"Total CPU time: {hours}h {minutes}m {seconds:.2f}s")


In [10]:
extract_cpu_time('Si/gga/bands/OUTCAR')

4.938

In [27]:
def band_data(material,dict):
    gga_bands_path = material+'/gga/bands/vasprun.xml'
    gga_bands_vasprun = Vasprun(gga_bands_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['GGA gap'] = gga_bands_vasprun.eigenvalue_band_properties[0]

In [28]:
def hse_static(material,dict,spin_polarised=True):
    hse_static_path = material+'/hse/static/vasprun.xml'
    hse_static_vasprun = Vasprun(hse_static_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['HSE static gap'] = hse_static_vasprun.eigenvalue_band_properties[0]

In [29]:
def hse_few(material,dict,spin_polarised=True):
    hse_few_path = material+'/hse/gap/vasprun.xml'
    hse_few_vasprun = Vasprun(hse_few_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['HSE few gap'] = hse_few_vasprun.eigenvalue_band_properties[0]

In [30]:
def shifted_optics(material,dict,energies):
    gga_optics_path = material+'/gga/optics/vasprun.xml'
    gga_optics_vasprun = Vasprun(gga_optics_path,parse_potcar_file=False)
    gga_alphas = gga_optics_vasprun.optical_absorption_coeff
    corr = dict[material]['HSE static gap'] - dict[material]['GGA gap']
    gga_energies = gga_optics_vasprun.dielectric[0]
    gga_interp_alphas = np.interp(energies,gga_energies,gga_alphas)
    shifted_alphas = shifter(gga_interp_alphas,corr)
    shifted_SLME = eff.calculate(energies,shifted_alphas)
    dict[material]['GGA + corr SLME'] = shifted_SLME
    return gga_interp_alphas, shifted_alphas

In [31]:
def hse_optics(material,dict,energies):
    hse_optics_path = material+'/hse/optics/vasprun.xml'
    hse_optics_vasprun = Vasprun(hse_optics_path,parse_potcar_file=False)
    hse_alphas = hse_optics_vasprun.optical_absorption_coeff
    hse_energies = hse_optics_vasprun.dielectric[0]
    hse_interp_alphas = np.interp(energies,hse_energies,hse_alphas)
    hse_SLME = eff.calculate(energies,hse_interp_alphas)
    dict[material]['HSE SLME'] = hse_SLME
    return hse_interp_alphas

In [32]:
def gga_bands_time(material,dict):
    gga_bands_path = material+'/gga/bands/OUTCAR'
    time = extract_cpu_time(gga_bands_path)
    dict[material]['GGA bands time'] = time

In [33]:
def gga_static_time(material,dict):
    gga_static_path = material+'/gga/static/OUTCAR'
    time = extract_cpu_time(gga_static_path)
    dict[material]['GGA static time'] = time

In [34]:
def gga_optics_time(material,dict):
    gga_optics_path = material+'/gga/optics/OUTCAR'
    time = extract_cpu_time(gga_optics_path)
    dict[material]['GGA optics time'] = time

In [35]:
def hse_static_time(material,dict):
    hse_static_path = material+'/hse/static/OUTCAR'
    time = extract_cpu_time(hse_static_path)
    dict[material]['HSE static time'] = time

In [36]:
def hse_optics_time(material,dict):
    hse_optics_path = material+'/hse/optics/OUTCAR'
    time = extract_cpu_time(hse_optics_path)
    dict[material]['HSE optics time'] = time

In [37]:
def hse_few_time(material,dict):
    hse_few_path = material+'/hse/gap/OUTCAR'
    time = extract_cpu_time(hse_few_path)
    dict[material]['HSE few time'] = time

In [38]:
def all_fields(material,dict):
    energies = np.linspace(0,5,101)
    try:
        band_data(material,dict)
    except:
        print(f"Error in {material} band data")
    try:
        hse_static(material,dict)
    except:
        print(f"Error in {material} HSE static")
    try:
        hse_few(material,dict)
    except:
        print(f"Error in {material} HSE few")
    try:
        gga_alphas, shifted_alphas = shifted_optics(material,dict,energies)
    except:
        print(f"Error in {material} optics")
    try:
        hse_alphas = hse_optics(material,dict,energies)
    except:
        print(f"Error in {material} HSE optics")
    fig,ax = plt.subplots(figsize=(5,5))
    ax.plot(energies,gga_alphas,label = 'GGA')
    ax.plot(energies,hse_alphas,label = 'HSE')
    ax.plot(energies,shifted_alphas,label = 'Shifted GGA')
    fig.legend()
    ax.set_xlabel('Energy (eV)')
    ax.set_ylabel(r'$\alpha$ / cm$^{-1}$')
    try:
        gga_static_time(material,dict)
    except:
        print(f"Error in {material} GGA static time")
    try:
        gga_bands_time(material,dict)
    except:
        print(f"Error in {material} GGA bands time")
    try:
        gga_optics_time(material,dict)
    except:
        print(f"Error in {material} GGA optics time")
    try:
        hse_static_time(material,dict)
    except:
        print(f"Error in {material} HSE static time")
    try:
        hse_optics_time(material,dict)
    except:
        print(f"Error in {material} HSE optics time")
    try:
        hse_few_time(material,dict)
    except:
        print(f"Error in {material} HSE few time")
    print(f"Following entries altered for {material} in dictionary", dict[material])
    return fig

In [39]:
key_materials = loadjson('key_materials.json')

In [40]:
data = {key: {} for key in key_materials}

In [41]:
fig = all_fields('Si', data)

Following entries altered for Si in dictionary {'GGA gap': 0.5130999999999997, 'HSE static gap': 1.1666999999999996, 'HSE few gap': 1.7303000000000006, 'GGA + corr SLME': 3.1583337550403137, 'HSE SLME': 1.887657797722469, 'GGA static time': 8.323, 'GGA bands time': 4.938, 'GGA optics time': 5.556, 'HSE static time': 189.814, 'HSE optics time': 173.962, 'HSE few time': 15.988}


In [42]:
missing = []
for material in key_materials:
    try:
        fig = all_fields(material,data)
        fig.savefig(f'abs_plots/{material}.pdf')
    except:
        print(f"Missing data for {material}")
        missing.append(material)

Following entries altered for GaAs in dictionary {'GGA gap': 0.42019999999999946, 'HSE static gap': 1.3794999999999997, 'HSE few gap': 1.6846000000000005, 'GGA + corr SLME': 29.40895389040265, 'HSE SLME': 25.01346913470742, 'GGA static time': 21.555, 'GGA bands time': 98.181, 'GGA optics time': 16.543, 'HSE static time': 545.494, 'HSE optics time': 702.546, 'HSE few time': 47.27}
Following entries altered for InP in dictionary {'GGA gap': 0.4942000000000002, 'HSE static gap': 1.3791000000000002, 'HSE few gap': 2.684, 'GGA + corr SLME': 25.418717082083507, 'HSE SLME': 23.03413934898326, 'GGA static time': 12.566, 'GGA bands time': 5.735, 'GGA optics time': 7.056, 'HSE static time': 231.889, 'HSE optics time': 213.787, 'HSE few time': 69.936}
Following entries altered for CdTe in dictionary {'GGA gap': 0.5479999999999998, 'HSE static gap': 1.4345, 'HSE few gap': 1.9522, 'GGA + corr SLME': 24.910146037969067, 'HSE SLME': 22.703353067513564, 'GGA static time': 13.908, 'GGA bands time': 6.5

In [43]:
for material in key_materials:
    if material in missing:
        try:
            fig = all_fields(material,data)
            fig.savefig(f'abs_plots/{material}.pdf')
            missing.remove(material)
        except:
            print(f"Missing data for {material}")

In [44]:
missing

[]

In [46]:
headers = list(data["InP"].keys())

In [47]:
df = pd.DataFrame(data).T

In [ ]:
df.head()

,GGA gap,HSE static gap,HSE few gap,GGA + corr SLME,HSE SLME,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time,HSE few time
GaAs,0.4202,1.3795,1.6846,29.408954,25.013469,21.555,98.181,16.543,545.494,702.546,47.270
InP,0.4942,1.3791,2.6840,25.418717,23.034139,12.566,5.735,7.056,231.889,213.787,69.936
CdTe,0.5480,1.4345,1.9522,24.910146,22.703353,13.908,6.577,7.789,252.441,190.582,53.780
MAPbI3,1.6834,2.3827,2.3830,11.455984,13.881105,121.952,130.802,198.756,596.506,505.214,136.774
GaCuSe2,0.1252,1.3668,1.7600,27.734243,22.006432,29.675,20.594,35.820,749.109,635.948,39.792
InCuSe2,0.0095,0.7630,1.1702,10.375573,23.560902,33.098,26.208,41.523,786.352,624.712,40.261
ZnCu2SnS4,0.1509,1.3468,1.7550,28.697312,23.637190,32.064,24.935,50.326,4583.033,3878.417,38.187
CuSbS2,0.7096,1.6112,1.5342,25.693751,26.452798,49.224,43.890,105.457,5219.081,2842.610,170.495
Sb2S3,1.2561,1.8564,1.7479,22.325574,19.452758,78.297,42.645,85.909,14583.390,12362.016,428.321
Cu2O,0.5258,2.0653,2.3741,5.555936,7.780332,13.429,7.060,13.238,378.456,212.456,94.180


In [49]:
avg_row = df.mean(axis=0)

In [ ]:
time_emit = 40931.14848
emit_en = 0.237588728

In [51]:
avg_row[5:]

GGA static time      41.333923
GGA bands time       37.169385
GGA optics time      55.148615
HSE static time    2546.362308
HSE optics time    2032.267769
HSE few time        128.493462
dtype: float64

In [52]:
calcs = ["GGA static", "GGA bands", "GGA optics", "HSE static", "HSE optics", "HSE few"]

In [53]:
times_dict = {time:calc for calc,time in zip(avg_row[5:],calcs)}

In [54]:
times_dict

{'GGA static': 41.33392307692308,
 'GGA bands': 37.16938461538461,
 'GGA optics': 55.148615384615404,
 'HSE static': 2546.3623076923072,
 'HSE optics': 2032.2677692307693,
 'HSE few': 128.49346153846156}

In [ ]:
times_dict['ML'] = 0.018581
times_dict['HSE bands'] = times_dict['HSE static'] + times_dict['GGA bands']

In [57]:
dumpjson(times_dict,'Calc_times.json')

In [65]:
column_indices = slice(5,-1,1)

In [69]:
column_names = df.columns[column_indices]

In [84]:
df_2 = df[column_names].copy()

In [85]:
df_2

,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time
GaAs,21.555,98.181,16.543,545.494,702.546
InP,12.566,5.735,7.056,231.889,213.787
CdTe,13.908,6.577,7.789,252.441,190.582
MAPbI3,121.952,130.802,198.756,596.506,505.214
GaCuSe2,29.675,20.594,35.820,749.109,635.948
InCuSe2,33.098,26.208,41.523,786.352,624.712
ZnCu2SnS4,32.064,24.935,50.326,4583.033,3878.417
CuSbS2,49.224,43.890,105.457,5219.081,2842.610
Sb2S3,78.297,42.645,85.909,14583.390,12362.016
Cu2O,13.429,7.060,13.238,378.456,212.456


In [86]:
df_2['HSE bands time'] = df_2['HSE static time'] + df_2['GGA bands time']

In [87]:
df_2

,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time,HSE bands time
GaAs,21.555,98.181,16.543,545.494,702.546,643.675
InP,12.566,5.735,7.056,231.889,213.787,237.624
CdTe,13.908,6.577,7.789,252.441,190.582,259.018
MAPbI3,121.952,130.802,198.756,596.506,505.214,727.308
GaCuSe2,29.675,20.594,35.820,749.109,635.948,769.703
InCuSe2,33.098,26.208,41.523,786.352,624.712,812.560
ZnCu2SnS4,32.064,24.935,50.326,4583.033,3878.417,4607.968
CuSbS2,49.224,43.890,105.457,5219.081,2842.610,5262.971
Sb2S3,78.297,42.645,85.909,14583.390,12362.016,14626.035
Cu2O,13.429,7.060,13.238,378.456,212.456,385.516


In [88]:
for col in df_2.columns:
    en_name = col[:-5]+' energy / kWh'
    co2_name = col[:-5]+' CO2 use / kg'
    df_2[co2_name] = df_2[col] / time_emit
    df_2[en_name] = df_2[co2_name] / emit_en

In [89]:
df_2

,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time,HSE bands time,GGA static CO2 use / kg,GGA static energy / kWh,GGA bands CO2 use / kg,GGA bands energy / kWh,GGA optics CO2 use / kg,GGA optics energy / kWh,HSE static CO2 use / kg,HSE static energy / kWh,HSE optics CO2 use / kg,HSE optics energy / kWh,HSE bands CO2 use / kg,HSE bands energy / kWh
GaAs,21.555,98.181,16.543,545.494,702.546,643.675,0.000527,0.002217,0.002399,0.010096,0.000404,0.001701,0.013327,0.056093,0.017164,0.072243,0.015726,0.066189
InP,12.566,5.735,7.056,231.889,213.787,237.624,0.000307,0.001292,0.000140,0.000590,0.000172,0.000726,0.005665,0.023845,0.005223,0.021984,0.005805,0.024435
CdTe,13.908,6.577,7.789,252.441,190.582,259.018,0.000340,0.001430,0.000161,0.000676,0.000190,0.000801,0.006167,0.025959,0.004656,0.019598,0.006328,0.026635
MAPbI3,121.952,130.802,198.756,596.506,505.214,727.308,0.002979,0.012540,0.003196,0.013450,0.004856,0.020438,0.014573,0.061339,0.012343,0.051951,0.017769,0.074789
GaCuSe2,29.675,20.594,35.820,749.109,635.948,769.703,0.000725,0.003051,0.000503,0.002118,0.000875,0.003683,0.018302,0.077031,0.015537,0.065395,0.018805,0.079149
InCuSe2,33.098,26.208,41.523,786.352,624.712,812.560,0.000809,0.003403,0.000640,0.002695,0.001014,0.004270,0.019212,0.080861,0.015263,0.064239,0.019852,0.083556
ZnCu2SnS4,32.064,24.935,50.326,4583.033,3878.417,4607.968,0.000783,0.003297,0.000609,0.002564,0.001230,0.005175,0.111969,0.471274,0.094755,0.398818,0.112579,0.473838
CuSbS2,49.224,43.890,105.457,5219.081,2842.610,5262.971,0.001203,0.005062,0.001072,0.004513,0.002576,0.010844,0.127509,0.536679,0.069449,0.292306,0.128581,0.541192
Sb2S3,78.297,42.645,85.909,14583.390,12362.016,14626.035,0.001913,0.008051,0.001042,0.004385,0.002099,0.008834,0.356291,1.499611,0.302020,1.271187,0.357333,1.503997
Cu2O,13.429,7.060,13.238,378.456,212.456,385.516,0.000328,0.001381,0.000172,0.000726,0.000323,0.001361,0.009246,0.038917,0.005191,0.021847,0.009419,0.039643


In [92]:
df2_rounded = df_2.round(4)

In [93]:
df2_rounded

,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time,HSE bands time,GGA static CO2 use / kg,GGA static energy / kWh,GGA bands CO2 use / kg,GGA bands energy / kWh,GGA optics CO2 use / kg,GGA optics energy / kWh,HSE static CO2 use / kg,HSE static energy / kWh,HSE optics CO2 use / kg,HSE optics energy / kWh,HSE bands CO2 use / kg,HSE bands energy / kWh
GaAs,21.555,98.181,16.543,545.494,702.546,643.675,0.0005,0.0022,0.0024,0.0101,0.0004,0.0017,0.0133,0.0561,0.0172,0.0722,0.0157,0.0662
InP,12.566,5.735,7.056,231.889,213.787,237.624,0.0003,0.0013,0.0001,0.0006,0.0002,0.0007,0.0057,0.0238,0.0052,0.0220,0.0058,0.0244
CdTe,13.908,6.577,7.789,252.441,190.582,259.018,0.0003,0.0014,0.0002,0.0007,0.0002,0.0008,0.0062,0.0260,0.0047,0.0196,0.0063,0.0266
MAPbI3,121.952,130.802,198.756,596.506,505.214,727.308,0.0030,0.0125,0.0032,0.0135,0.0049,0.0204,0.0146,0.0613,0.0123,0.0520,0.0178,0.0748
GaCuSe2,29.675,20.594,35.820,749.109,635.948,769.703,0.0007,0.0031,0.0005,0.0021,0.0009,0.0037,0.0183,0.0770,0.0155,0.0654,0.0188,0.0791
InCuSe2,33.098,26.208,41.523,786.352,624.712,812.560,0.0008,0.0034,0.0006,0.0027,0.0010,0.0043,0.0192,0.0809,0.0153,0.0642,0.0199,0.0836
ZnCu2SnS4,32.064,24.935,50.326,4583.033,3878.417,4607.968,0.0008,0.0033,0.0006,0.0026,0.0012,0.0052,0.1120,0.4713,0.0948,0.3988,0.1126,0.4738
CuSbS2,49.224,43.890,105.457,5219.081,2842.610,5262.971,0.0012,0.0051,0.0011,0.0045,0.0026,0.0108,0.1275,0.5367,0.0694,0.2923,0.1286,0.5412
Sb2S3,78.297,42.645,85.909,14583.390,12362.016,14626.035,0.0019,0.0081,0.0010,0.0044,0.0021,0.0088,0.3563,1.4996,0.3020,1.2712,0.3573,1.5040
Cu2O,13.429,7.060,13.238,378.456,212.456,385.516,0.0003,0.0014,0.0002,0.0007,0.0003,0.0014,0.0092,0.0389,0.0052,0.0218,0.0094,0.0396


In [94]:
df2_rounded.to_csv('VASP_carbon_data.csv')